In [1]:
!git clone https://github.com/sunandhini96/TSAI_ERAV1.git

Cloning into 'TSAI_ERAV1'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 557 (delta 55), reused 24 (delta 24), pack-reused 472
Receiving objects: 100% (557/557), 14.04 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (247/247), done.


In [2]:
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
cd /kaggle/working/TSAI_ERAV1/Session_17

/kaggle/working/TSAI_ERAV1/Session_17


In [5]:
from datamodules.bert_datamodule import *

In [6]:
import os
from transformer import *
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [8]:
# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

initializing..


In [9]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = '/kaggle/input/training/training.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = 'vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)



loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...


In [10]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code=n_code, n_heads=n_heads, embed_size=embed_size, inner_ff_size=inner_ff_size, n_embeddings=len(dataset.vocab), seq_len=seq_len, dropout=dropout,algorithm="BERT")
model=model.cuda()


# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 10000
for it in range(n_iteration):

    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)

    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)

    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_each == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()



initializing model...
initializing optimizer and loss...
training...
it: 0  | loss 10.25  | Δw: 1.172
it: 10  | loss 9.5  | Δw: 0.227
it: 20  | loss 9.32  | Δw: 0.137
it: 30  | loss 9.17  | Δw: 0.111
it: 40  | loss 9.05  | Δw: 0.109
it: 50  | loss 8.86  | Δw: 0.103
it: 60  | loss 8.69  | Δw: 0.1
it: 70  | loss 8.57  | Δw: 0.097
it: 80  | loss 8.39  | Δw: 0.096
it: 90  | loss 8.2  | Δw: 0.098
it: 100  | loss 8.07  | Δw: 0.097
it: 110  | loss 7.9  | Δw: 0.102
it: 120  | loss 7.78  | Δw: 0.1
it: 130  | loss 7.68  | Δw: 0.108
it: 140  | loss 7.57  | Δw: 0.11
it: 150  | loss 7.39  | Δw: 0.14
it: 160  | loss 7.33  | Δw: 0.128
it: 170  | loss 7.27  | Δw: 0.111
it: 180  | loss 7.18  | Δw: 0.112
it: 190  | loss 7.06  | Δw: 0.127
it: 200  | loss 6.98  | Δw: 0.136
it: 210  | loss 6.86  | Δw: 0.182
it: 220  | loss 6.8  | Δw: 0.148
it: 230  | loss 6.82  | Δw: 0.139
it: 240  | loss 6.72  | Δw: 0.198
it: 250  | loss 6.72  | Δw: 0.158
it: 260  | loss 6.59  | Δw: 0.177
it: 270  | loss 6.65  | Δw: 0.219

In [12]:

# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )
print('end')





saving embeddings...
end
